In [ ]:
import json

# Open and read the JSON file
with open('/aia_chunks_querverweise_index_with_text_length.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print the data
data = data['chunks']

In [118]:
from pyvis.network import Network
import networkx as nx
import json

# Create a mapping of id to title
id_to_title = {entry["id"]: entry["title"] for entry in data}

# Build a NetworkX graph to calculate degrees
G = nx.DiGraph()

# Add nodes to the graph
for entry in data:
    G.add_node(entry["title"])

# Add edges to the graph
for entry in data:
    source_title = entry["title"]
    for target_id in entry["relevantChunksIds"]:
        target_title = id_to_title.get(target_id)
        if target_title:  # Ensure target node exists
            G.add_edge(source_title, target_title)

# Function to determine node color based on title prefix
def determine_color(title):
    if title.startswith("ErwG"):
        return "violet"
    elif title.startswith("Art"):
        return "SpringGreen"
    elif title.startswith("Anhang"):
        return "SkyBlue"
    else:
        return "gray"

# Initialize a Pyvis network with remote resources for compatibility
net = Network(notebook=True, directed=True, height="900px", width="100%", cdn_resources="remote", select_menu=True, filter_menu=True)

# Create a dictionary for fast content lookup by title
title_to_content = {entry['title']: entry['content'] for entry in data}

for node in G.nodes:
    color = determine_color(node)
    
    # Find content length based on title
    content_length = 0
    for entry in data:
        if entry['title'] == node:
            content_length = entry['content_length']
            break

    # Define size based on content length (e.g., scale it appropriately)
    size = 10 + content_length / 100  # You can adjust the scaling factor based on your data's content_length range
    
    # Fetch the content based on the node's title
    content = title_to_content.get(node, "No content available")

    # Create hover text with truncated content (if necessary)
    MAX_CONTENT_LENGTH = 3000  # Adjust the max length for hover content
    hover_text = f"Title: {node} | Content Length: {content_length} | {content[:MAX_CONTENT_LENGTH]}..."

    # Add node with hover text
    net.add_node(node, label=node, title=hover_text, color={"background": color, "border": "black"}, size=size)

# Add edges
for edge in G.edges:
    source = edge[0]
    target = edge[1]
    edge_color = determine_color(source)
    net.add_edge(source, target, title="relates to", color=edge_color, borderWidth=2, borderColor="black")

#net.toggle_physics(True)
#net.show_buttons(filter_=['physics'])
net.set_options("""
                const options = {
  "physics": {
    "repulsion": {
      "centralGravity": 0,
      "springLength": 380,
      "nodeDistance": 210
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}
                """)

# Show the graph
net.show("interactive_graph.html")


interactive_graph.html
